In [33]:
#%pip install --upgrade --user --quiet google-cloud-aiplatform
#%pip install google-generativeai
#%pip install google-auth-oauthlib

In [46]:
%pip install -U -q "google-genai"

Note: you may need to restart the kernel to use updated packages.


In [1]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=H9sNQEUvMJqtCVu2HJQJqNZzku68dr&access_type=offline&code_challenge=z_Jo0nBYmMf3P6YQhH_OIJ_CuprafBJ2dK-cqJIC66Y&code_challenge_method=S256


Credentials saved to file: [/Users/shepsheppard/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "gemini-shep" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [3]:
import typing
import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps

from vertexai.generative_models import (
    GenerationConfig,
    Image,
)

def gemini(prompt, model):
    responses = model.generate_content(prompt,
                                     stream=True)

    response_text = ""
    for response in responses:
        response_text += response.text

    return response_text

def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)

def print_multimodal_prompt(contents: list):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if isinstance(content, Image):
            display_images([content])
        elif isinstance(content, Part):
            url = get_url_from_gcs(content.file_data.file_uri)
            IPython.display.display(load_image_from_url(url))
        else:
            print(content)

def gemini_vision(contents_image, model):

    responses = model.generate_content(
        contents_image,
        stream=True)

    response_text = ""
    for response in responses:
        response_text += response.text
    return response_text

def gemini_vision_parameters(contents_image, model, config):

    responses = model.generate_content(
        contents=contents_image,
        generation_config=config,
        stream=True
    )

    response_text = ""
    for response in responses:
        response_text += response.text

    return response_text

In [4]:
PROJECT_ID = "gemini-shep"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai
from vertexai.generative_models import GenerativeModel

vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel("gemini-1.0-pro-002")


### Travel Use Case

In [16]:
prompt="Create list of 10 locations to visit with no internet, white sand beaches, no cars and is an island and accessible only by canoe:\n\n"
response_1 = gemini(prompt=prompt, model=model)


print(response_1)

## 10 Island Paradises with White Sand Beaches, No Cars, and No Internet, Accessible Only by Canoe:

1. **Cayo Saetía, Cuba:** This remote island boasts pristine white sand beaches, untouched natural beauty, and a diverse ecosystem with wild horses roaming freely. While you can reach the island by car, it's most enjoyable to explore via canoe, allowing you to discover hidden coves and witness the island's tranquility.

2. **Palawan Island, Philippines:** This archipelago features stunning limestone cliffs, turquoise waters, and secluded beaches like Nacpan Beach, only accessible by boat or a scenic hike. With its untouched beauty and minimal development, Palawan offers a digital detox in a true island paradise.

3. **Holbox Island, Mexico:** This car-free island is a haven for eco-tourism, with flamingos, whale sharks, and bioluminescent plankton being some of the highlights. You can explore the island's pristine beaches and mangroves by bike or kayak, disconnecting from the digital wo

### Table Use Case

In [18]:
stock_prompt="Stock Performance Table:\nDate,Symbol,Adj Close,Close,High,Low,Open,Volume\n2009-12-31,MMM,,,,,,\n2010-01-04,MMM,59.318885803222656,83.0199966430664,83.44999694824219,82.66999816894531,83.08999633789062,3043700.0\n2010-01-05,MMM,58.94734191894531,82.5,83.2300033569336,81.69999694824219,82.80000305175781,2847000.0\n2010-01-06,MMM,59.783294677734375,83.66999816894531,84.5999984741211,83.51000213623047,83.87999725341797,5268500.0\n2010-01-07,MMM,59.826175689697266,83.7300033569336,83.76000213623047,82.12000274658203,83.31999969482422,4470100.0\n2010-01-08,MMM,60.24774932861328,84.31999969482422,84.31999969482422,83.30000305175781,83.69000244140625,3405800.0\n2010-01-11,MMM,60.004825592041016,83.9800033569336,84.5999984741211,83.41000366210938,84.37999725341797,2927100.0\n2010-01-12,MMM,60.05484390258789,84.05000305175781,84.18000030517578,83.30000305175781,83.58000183105469,3031800.0\n2010-01-13,MMM,59.861942291259766,83.77999877929688,84.11000061035156,83.19999694824219,84.11000061035156,3102000.0\n2010-01-14,MMM,59.661865234375,83.5,83.93000030517578,83.41999816894531,83.73999786376953,2634100.0\n2010-01-15,MMM,59.56898880004883,83.37000274658203,84.08999633789062,82.87000274658203,83.5199966430664,3955000.0\n2010-01-19,MMM,60.819358825683594,85.12000274658203,85.16999816894531,83.5,83.81999969482422,4500400.0\n2010-01-20,MMM,60.5335578918457,84.72000122070312,85.12999725341797,83.58999633789062,84.83000183105469,3671200.0\n2010-01-21,MMM,59.090240478515625,82.69999694824219,84.5999984741211,82.56999969482422,84.5999984741211,4783200.0\n2010-01-22,MMM,58.218544006347656,81.4800033569336,82.83000183105469,81.30000305175781,82.4000015258789,4809000.0\n2010-01-25,MMM,58.4686164855957,81.83000183105469,82.88999938964844,81.4800033569336,82.33000183105469,3386600.0\n2010-01-26,MMM,58.32571792602539,81.62999725341797,82.7300033569336,81.04000091552734,81.45999908447266,3138000.0\n2010-01-27,MMM,58.80441665649414,82.30000305175781,82.83999633789062,81.01000213623047,81.33000183105469,5066900.0\n2010-01-28,MMM,57.69694519042969,80.75,82.63999938964844,79.11000061035156,82.62000274658203,6820700.0\n2010-01-29,MMM,57.51118087768555,80.48999786376953,81.87999725341797,80.18000030517578,81.3499984741211,4347000.0\n2010-02-01,MMM,57.47545623779297,80.44000244140625,80.88999938964844,79.94000244140625,80.83999633789062,3632700.0\n2010-02-02,MMM,57.46830368041992,80.43000030517578,80.94000244140625,79.69999694824219,80.69999694824219,4690000.0\n2010-02-03,MMM,58.23284912109375,81.5,81.68000030517578,79.83000183105469,79.83000183105469,3401300.0\n2010-02-04,MMM,56.59660720825195,79.20999908447266,81.12999725341797,78.83000183105469,81.12999725341797,5312600.0\n2010-02-05,MMM,56.11787796020508,78.54000091552734,79.5,77.26000213623047,79.16000366210938,5408000.0\n2010-02-08,MMM,55.39621353149414,77.52999877929688,78.4800033569336,77.25,78.4800033569336,4407300.0\n2010-02-09,MMM,56.246490478515625,78.72000122070312,79.41000366210938,77.9000015258789,78.27999877929688,4252500.0\n2010-02-10,MMM,56.175025939941406,78.62000274658203,79.29000091552734,78.0,78.80000305175781,2445200.0\n2010-02-11,MMM,57.35396194458008,80.2699966430664,80.38999938964844,78.83999633789062,79.22000122070312,5524500.0\n2010-02-12,MMM,56.575157165527344,79.18000030517578,79.2300033569336,78.06999969482422,79.13999938964844,5443000.0\n2010-02-16,MMM,57.496891021728516,80.47000122070312,80.63999938964844,79.33999633789062,80.08000183105469,3619100.0\n2010-02-17,MMM,57.63711929321289,80.13999938964844,80.69000244140625,79.73999786376953,80.41999816894531,3095200\n"
print(stock_prompt)


Stock Performance Table:
Date,Symbol,Adj Close,Close,High,Low,Open,Volume
2009-12-31,MMM,,,,,,
2010-01-04,MMM,59.318885803222656,83.0199966430664,83.44999694824219,82.66999816894531,83.08999633789062,3043700.0
2010-01-05,MMM,58.94734191894531,82.5,83.2300033569336,81.69999694824219,82.80000305175781,2847000.0
2010-01-06,MMM,59.783294677734375,83.66999816894531,84.5999984741211,83.51000213623047,83.87999725341797,5268500.0
2010-01-07,MMM,59.826175689697266,83.7300033569336,83.76000213623047,82.12000274658203,83.31999969482422,4470100.0
2010-01-08,MMM,60.24774932861328,84.31999969482422,84.31999969482422,83.30000305175781,83.69000244140625,3405800.0
2010-01-11,MMM,60.004825592041016,83.9800033569336,84.5999984741211,83.41000366210938,84.37999725341797,2927100.0
2010-01-12,MMM,60.05484390258789,84.05000305175781,84.18000030517578,83.30000305175781,83.58000183105469,3031800.0
2010-01-13,MMM,59.861942291259766,83.77999877929688,84.11000061035156,83.19999694824219,84.11000061035156,3102000.0

In [20]:
gemini(prompt=stock_prompt, model=model)

"## Analysis of Stock Performance Table for MMM in 2010\n\nThis table represents the daily stock performance of MMM for the first two months of 2010. We can analyze this data in multiple ways depending on your interests. Here are a few ways we can analyze this data:\n\n**1. Overall Price Movement:**\n\n* The **opening price** for MMM on January 4th, 2010 was $83.02. By February 17th, 2010, the **closing price** had decreased to $80.14, representing a **decline of approximately 3.44%**.\n* However, during this timeframe, there were also periods of increases. Between January 19th and January 20th, the closing price increased by approximately 1.6%.\n\n**2. Volatility:**\n\n* The data demonstrates some **volatility** in the stock price of MMM during this period. The highest **daily high** was $85.17 on January 19th, while the lowest **daily low** was $77.26 on February 5th. This represents a difference of nearly $8 per share.\n* Additionally, the **volume** of shares traded varied consider

In [22]:
response_1 = model.generate_content(stock_prompt, stream=True)
for response in response_1:
    print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "##"
    }
  }
}
usage_metadata {
}
model_version: "gemini-1.0-pro-002"
create_time {
  seconds: 1740233619
  nanos: 113090000
}
response_id: "k9u5Z8LzBp6Bm9IPp9Sb8Qw"

candidates {
  content {
    role: "model"
    parts {
      text: " Stock Performance Analysis for MMM in January 2010\n\nThis analysis examines the"
    }
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.2109375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.181640625
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.150390625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.06640625
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.18359375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.145507812
  }
  safety_ra

In [26]:
prompt = "{}\n{}".format(stock_prompt,'What did MMM close on 2010-01-20?')

response_1 = model.generate_content(prompt, stream=True)

for response in response_1:
    print(response.text)



The
 stock MMM closed on 2010-01-20 at
 **$84.72**. 

This information can be found on the
 second line of the table under the `Close` column for the date `2010-01-20`. 



### Contract Use Case

In [28]:
contract_prompt = 'Definitions: I agree to, acknowledge, and represent the following: • "Lender" includes the Lender’s agents, service providers, and any of their successors and assigns. • "Other Loan Participants" includes (i) any actual or potential owners of a loan resulting from this application (the “Loan”), (ii) acquirers of any beneficial or other interest in the Loan, (iii) any mortgage insurer, (iv) any guarantor, (v) any servicer of the Loan, and (vi) any of these parties service providers, successors or assigns. (1) The Complete Information for this Application • The information I have provided in this application is true, accurate, and complete as of the date I signed this application. • If the information I submitted changes or I have new information before closing of the Loan, I must change and supplement this application, including providing any updated/supplemented real estate sales contract. • For purchase transactions: The terms and conditions of any real estate sales contract signed by me in connection with this application are true, accurate, and complete to the best of my knowledge and belief. I have not entered into any other agreement, written or oral, in connection with this real estate transaction. • The Lender and Other Loan Participants may rely on the information contained in the application before and after closing of the Loan. • Any intentional or negligent misrepresentation of information may result in the imposition of: (a) civil liability on me, including monetary damages, if a person suffers any loss because the person relied on any misrepresentation that I have made on this application, and/or (b) criminal penalties on me including, but not limited to, fine or imprisonment or both under the provisions of Federal law (18 U.S.C. §§ 1001 et seq.). (2) The Property’s Security The Loan I have applied for in this application will be secured by a mortgage or deed of trust which provides the Lender a security interest in the property described in this application. (3) The Property’s Appraisal, Value, and Condition • Any appraisal or value of the property obtained by the Lender is for use by the Lender and Other Loan Participants. • The Lender and Other Loan Participants have not made any representation or warranty, express or implied, to me about the property, its condition, or its value. (4) Electronic Records and Signatures • The Lender and Other Loan Participants may keep any paper record and/or electronic record of this application, whether or not the Loan is approved. • If this application is created as (or converted into) an “electronic application”, I consent to the use of “electronic records” and “electronic signatures” as the terms are defined in and governed by applicable Federal and/or state electronic transactions laws. • I intend to sign and have signed this application either using my: (a) electronic signature; or (b) a written signature and agree that if a paper version of this application is converted into an electronic application, the application will be an electronic record, and the representation of my written signature on this application will be my binding electronic signature. • I agree that the application, if delivered or transmitted to the Lender or Other Loan Participants as an electronic record with my electronic signature, will be as effective and enforceable as a paper application signed by me in writing. (5) Delinquency • The Lender and Other Loan Participants may report information about my account to credit bureaus. Late payments, missed payments, or other defaults on my account may be reflected in my credit report and will likely affect my credit score. • If I have trouble making my payments I understand that I may contact a HUD-approved housing counseling organization for advice about actions I can take to meet my mortgage obligations. (6) Authorization for Use and Sharing of Information By signing below, in addition to the representations and agreements made above, I expressly authorize the Lender and Other Loan Participants to obtain, use, and share with each other (i) the loan application and related loan information and documentation, (ii) a consumer credit report on me, and (iii) my tax return information, as necessary to perform the actions listed below, for so long as they have an interest in my loan or its servicing: (a) process and underwrite my loan; (b) verify any data contained in my consumer credit report, my loan application and other information supporting my loan application; (c) inform credit and investment decisions by the Lender and Other Loan Participants; (d) perform audit, quality control, and legal compliance analysis and reviews; (e) perform analysis and modeling for risk assessments; (f) monitor the account for this loan for potential delinquencies and determine any assistance that may be available to me; and (g) other actions permissible under applicable law.'
#print(contract_prompt)

In [32]:
contract_prompt = "{}\n{}".format('this is a contract, What happens if I miss a payment?  ',contract_prompt)

response_1 = model.generate_content(contract_prompt, stream=True)
for response in response_1:
    print(response.text)


##
 Missed Payments

This document outlines the consequences of missing a payment on the loan you
 are applying for. 

**Consequences of Missed Payments:**

* **Late
 Fees:** The lender may charge late fees for missed payments. The specific amount of the late fee will be outlined in your loan agreement.
* **Reported to Credit Bure
aus:** Missed or late payments may be reported to credit bureaus, which could negatively impact your credit score. A lower credit score can make it more difficult to obtain loans
 and other forms of credit in the future, and may also lead to higher interest rates. 
* **Default:** If you miss multiple payments, your loan may be considered in default. This could result in the lender taking action to collect the debt,
 including:
    * **Acceleration:** The lender may demand immediate payment of the entire outstanding balance of the loan.
    * **Foreclosure:** If the loan is secured by a mortgage on your home, the lender may foreclose on
 the property and sell 

In [34]:
prompt = "{}\n{}".format(contract_prompt,'What can the lender do with my information?')
response_1 = model.generate_content(prompt, stream=True)
for response in response_1:
    print(response.text)

##
 Lender Rights in Case of Missed Payments

The provided excerpt details the consequences of missing
 a payment on your loan. Here's a breakdown of the key points:


**Missed Payments and Credit Reporting:**

* Late or missed payments will be reported to credit bureaus, potentially impacting your credit score.
* This information will be
 accessible to other lenders and financial institutions.

**Delinquency and Consequences:**

* Missed payments may be considered a delinquency, which could lead to additional fees
 and penalties.
* The lender has the right to accelerate the loan, meaning they can demand immediate repayment of the entire remaining balance.
* In extreme cases, the lender may initiate foreclosure proceedings to seize the property used as collateral for the loan.


**Options for Avoiding Delinquency:**

* Contact the lender as soon as possible if you anticipate difficulty making payments.
* Explore options for modifying your loan terms or seeking assistance from a HUD-app

### Call Use Case

In [36]:
call_prompt= "Transcript:Hi, how's it going? Yes, I've been looking at My Portfolio and. Yeah, the performance has been underperforming. It's just been bad. It's been underperforming all the major indexes. And I just want to know what you have me into. Yes, this is John Doe, yes ending in 7243. Ok yea can we keep going? Well, what's the outlook for the next five years? Because, um, I'm, you know? Looking to get this turned around. I know we're in a recession, but. I barely holding cash right now. Well, I mean, what about low risk will we have for that? You know, to be honest, I'd, I'd I'd be into looking at, um, bonds or or. I mean, my target date funds are killing me right now. I don't know what's gonna happen there. OK. Yeah, what can we do? I mean, I. I think I need some cash soon. Um plan to make a big purchase where, where can we realize the losses? Alright, Yep, just sell it. Yeah, just find me somewhere. I need about 200 grand. I I just need to make it happen. Alright, thanks. Bye."
print(call_prompt)

Transcript:Hi, how's it going? Yes, I've been looking at My Portfolio and. Yeah, the performance has been underperforming. It's just been bad. It's been underperforming all the major indexes. And I just want to know what you have me into. Yes, this is John Doe, yes ending in 7243. Ok yea can we keep going? Well, what's the outlook for the next five years? Because, um, I'm, you know? Looking to get this turned around. I know we're in a recession, but. I barely holding cash right now. Well, I mean, what about low risk will we have for that? You know, to be honest, I'd, I'd I'd be into looking at, um, bonds or or. I mean, my target date funds are killing me right now. I don't know what's gonna happen there. OK. Yeah, what can we do? I mean, I. I think I need some cash soon. Um plan to make a big purchase where, where can we realize the losses? Alright, Yep, just sell it. Yeah, just find me somewhere. I need about 200 grand. I I just need to make it happen. Alright, thanks. Bye.


In [38]:
prompt = "Generate detailed call summary notes from text\nText:\n###{}###\n".format(call_prompt)
response_1 = model.generate_content(prompt, stream=True)
for response in response_1:
    print(response.text)

##
 Call Summary Notes

**Caller:** John Doe (ending in 7243)


**Date:** Unidentified

**Summary:**

* Caller is concerned about the under
performance of their portfolio.
* The portfolio has been underperforming major indexes.
* Caller wants to know the outlook for the next five years.
*
 Caller is looking to get cash soon for a large purchase.
* Caller wants to realize losses.
* Caller needs about $200,000
.

**Action Items:**

* Review the caller's portfolio and provide an explanation for its underperformance.
* Discuss the outlook for the next five years and potential investment strategies.
* Explore options for realizing losses and generating cash.

* Identify potential investments that could meet the caller's needs.

**Additional Notes:**

* The caller is anxious and frustrated about the performance of their portfolio.
* The caller is open to exploring different investment options.
* It
 is important to address the caller's concerns and provide clear and concise information.

**N

In [39]:
prompt = "Generate list of topics discussed from text\nText:\n###{}###\n".format(call_prompt)

response_1 = model.generate_content(prompt, stream=True)

for response in response_1:
    print(response.text)

##
 Topics Discussed:

* **Portfolio Performance:** The client is concerned about the under
performance of their portfolio, which has been lagging behind major indexes.
* **
Outlook:** The client wants to know the outlook for their portfolio over the next five years, especially in light of the current recession.
* **Risk Tolerance:** The
 client expresses a low risk tolerance and is looking for options with lower risk.
* **Investment Options:** The client is interested in exploring bonds as an alternative to
 their underperforming target date funds.
* **Cash Needs:** The client needs to access a significant amount of cash soon for a large purchase.
* **Loss Realization:** The client wants to sell some of their investments to realize losses and
 free up cash. 
* **Target Amount:** The client needs to raise $200,000. 



In [44]:
prompt = "Generate list of follow-up tasks from text that the agent can perform\nText:\n###{}###\n".format(call_prompt)

response_1 = model.generate_content(prompt, stream=True)

for response in response_1:
    print(response.text)

##
 Follow-up tasks for the agent:

**1. Investment portfolio review:**

* Analyze the client's current portfolio and identify underperforming investments. 
*
 Compare portfolio performance to major market indexes.
* Provide the client with a clear understanding of their current investment strategy and its risks.

**2. Investment outlook:**
*
 Discuss the client's financial goals and risk tolerance.
* Present investment options for the next five years that align with the client's goals and risk tolerance.

* Consider low-risk options like bonds to generate cash flow and protect capital.

**3. Tax implications:**
* Inform the client about the tax implications of selling investments at a loss.
* Discuss potential strategies to minimize tax liability
.

**4. Cash generation:**
*  Identify investments that can be sold to generate the $200,000 the client needs.
*  Discuss alternative options for generating cash, such as loans or lines of credit.


**5. Follow-up communication:**
*  Schedule

In [48]:
prompt = "List how caller was verified by agent \nText:\n###{}###\n".format(call_prompt)

response_1 = model.generate_content(prompt, stream=True)

for response in response_1:
    print(response.text)

##
 Caller Verification:

Based on the provided transcript, the caller was verified by the agent


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "SAFETY",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.14746094,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.15527344
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "MEDIUM",
      "blocked": true,
      "probability_score": 0.80078125,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.20703125
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.18945312,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.13476562
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "probabil

### Disgruntled Call Use Case

In [52]:
call2_prompt="Transcript: Finally I got I get through to somebody on the other end. I've been going through your automated call tree here for about 5 minutes and punching buttons and I finally get through to the human being. It's about time you guys are wasting my time. No, I you know, I am so ****** *** at at your company. I bought my device I plugged it in and smoke started coming out of it and I tried to call you to get a refund or a replacement and it takes me nearly 10 minutes just to get through to a human being. This is absolutely brutal. Can't believe you guys are still in business. Yeah, it's the model, uh, model 128. Yep, uh, my office is covered in soot and the dogs running around chasing its tail and that was pretty, pretty bad. I need the device because I train for a triathlon. Ohh. Ohh, you might triathlon as well. Ohh really. Well, how far do you run? And what about the swimming? All good and we're kind of a bike, do you have? Road bike. Ohh OK. Yeah, now I've been training for about. About two months now. Yeah. Male OK, yeah, it's the dog. Seems to have settled down and I can probably wipe up the smoothie. OK. Well, what are my options? OK, you think they you think the better? The Model 993 is better. Alright. Well, sure, if you want to send me out one of those and uh. Ah, and I guess I'll just. Throw the other one out. Well, I guess this stuff happens, yeah? Or you're gonna you're gonna FedEx it to me. Tomorrow? Well, that's so that's pretty good. Well, no, I appreciate it and I'm sorry I was yelling at you. I guess it's not your fault. Yep. No that's a good day. I appreciate it. You've uh you've helped me out here and I'm I'm pretty thankful. So yeah thank you for your your time and thanks for helping me out here and. I appreciate you getting getting this thing sorted out pretty fast. Yep. Well, thanks. Yeah. And good luck with your next race as well. Yep. Thank you so much. "
print(call2_prompt)

Transcript: Finally I got I get through to somebody on the other end. I've been going through your automated call tree here for about 5 minutes and punching buttons and I finally get through to the human being. It's about time you guys are wasting my time. No, I you know, I am so ****** *** at at your company. I bought my device I plugged it in and smoke started coming out of it and I tried to call you to get a refund or a replacement and it takes me nearly 10 minutes just to get through to a human being. This is absolutely brutal. Can't believe you guys are still in business. Yeah, it's the model, uh, model 128. Yep, uh, my office is covered in soot and the dogs running around chasing its tail and that was pretty, pretty bad. I need the device because I train for a triathlon. Ohh. Ohh, you might triathlon as well. Ohh really. Well, how far do you run? And what about the swimming? All good and we're kind of a bike, do you have? Road bike. Ohh OK. Yeah, now I've been training for about.

In [56]:
generation_config_1 = GenerationConfig(
    temperature=0.0,
    top_k=5,
)

In [64]:
from google.genai import types
from google import genai

In [70]:
prompt = "Generate detailed call summary notes from text\nText:\n###{}###\n".format(call2_prompt)
response_1 = model.generate_content(prompt,stream=True)

for response in response_1:
    print(response.text)

##
 Call Summary
**Date:** 2020-10-2
8
**Time:** 14:30 PST
**Customer:**
 Male, frustrated with his Model 128 device malfunctioning
**Agent:** Sympathetic and helpful
**Issue:** Smoke coming out of Model 12
8; customer wants replacement
**Resolution:** Agent suggests Model 993 as a better replacement; customer agrees to receive a Model 993 via FedEx
 the next day
**Notes:**
* Customer was initially very angry, yelling at the agent for the long wait time to speak with a human and the malfunctioning device.
* Customer's anger subsides after agent shows empathy and
 suggests a suitable resolution.
* Customer expresses gratitude towards the end of the call and wishes the agent luck on their next race. 



In [72]:
prompt = "List mitigations agent used in call from text\nText:\n###{}###\n".format(call2_prompt)
response_1 = model.generate_content(prompt,stream=True)

for response in response_1:
    print(response.text)

##
 De-escalation Techniques Used in the Call Transcript:

This conversation showcases several de-escal
ation techniques used by the agent to calm the customer down and resolve the issue effectively.
 Here's a breakdown:

**1. Active Listening and Empathy:**

* The agent acknowledges the customer's frustration: "I understand you're frustrated
, and I apologize for the long wait time."
* They show empathy by validating the customer's feelings: "It's understandable that you're
 upset about the smoke coming from your device."
* They actively listen by summarizing the customer's concerns: "So, you're looking for a replacement or refund for your Model 128 device?"

**2. Offering
 Choices and Control:**

* The agent offers the customer options: "We can send you a replacement Model 128 or a different model, the Model 993."
* They empower the customer by allowing them to choose
: "Which option would you prefer?"
* They validate the customer's choice: "That's a good choice. The Model 993 is a v